In [ ]:
#Hand tracking project
import cv2
import mediapipe as mp #for hand detection
import pyautogui # to set the system volume on macOS.
import subprocess # for executing system commands

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
#module contains functions that can be used to draw landmarks and connections on the detected hands in the webcam frames.
#These functions are helpful for visualizing the results of the hand detection process.
drawing_utils = mp.solutions.drawing_utils #This line imports the drawing_utils module from mediapipe.solutions and assigns it to the variable drawing_utils

# Set the initial volume level
current_volume = 50
alpha = 0.2  # Smoothing factor for the exponential moving average

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
                        current_volume = min(100, current_volume)
                    else:
                        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
                        current_volume = max(0, current_volume)

    # Show the current volume as a vertical bar on the webcam image
    volume_bar_height = int((current_volume / 100) * frame_height)
    cv2.rectangle(image, (frame_width - 50, frame_height), (frame_width - 30, frame_height - volume_bar_height),
                  (0, 255, 0), -1)

    # Display volume level text
    cv2.putText(image, f"Volume: {current_volume}", (frame_width - 100, frame_height - 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Set the volume on macOS using AppleScript
    volume_percent = int(current_volume)
    subprocess.run(['osascript', '-e', f'set volume output volume {volume_percent}'])

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()